In [ ]:
import numpy as np
import scipy as sp
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

In [ ]:
roho_folder = "/cluster/projects/nn9297k/ROHO160+"
init_file = "/InputData/Initial/roho160_init_25_layers.nc"
init_old_file = "/InputData/Initial/roho160_init_v2.nc"
new_file = "/InputData/Initial/roho160_init_25_layers+.nc"

In [ ]:
ds = xr.open_dataset(roho_folder + init_file)
ds_old = xr.open_dataset(roho_folder + init_old_file)

In [ ]:
s_w = np.linspace(-1, 0, num=26)
s_rho = np.linspace(-0.98, -0.02, num=25)
coords = {
    's_rho': s_rho,
    's_w': s_w,
}
new_ds = ds_old.interp(coords).astype(np.float32)

In [ ]:
new_ds.temp.isel(ocean_time=-1, s_rho=-1).plot()

In [ ]:
parameters = (
    ('temp', 'xi_rho', 8), ('salt', 'xi_rho', 34), ('zeta', 'xi_rho', -0.34), 
    ('u', 'xi_u', 0), ('ubar', 'xi_u', 0), 
    ('v', 'xi_v', 0), ('vbar', 'xi_v', 0)
    )
for name, dim, fill_value in parameters:
    new_ds[name] = new_ds[name].interpolate_na(dim=dim, method="linear", fill_value=fill_value)

In [ ]:
new_ds.salt.isel(ocean_time=-1, s_rho=-1).plot()

In [ ]:
date = '2017-12-06T00:00:00.000000000'

#### Zeta

In [ ]:
variable = 'zeta'
plt.figure(figsize=(15, 7))
plt.subplot(121)
ds.sel(ocean_time=date)[variable].plot()
# result = sp.ndimage.gaussian_filter(
#     ds.isel(ocean_time=-1)[variable].values, 3, mode='nearest')
result = (ds.sel(ocean_time=date)[variable] * ds.mask_rho).values
# result = sp.ndimage.gaussian_filter(result, 3, mode='nearest')
# result = result * ds.mask_rho.values
ds = ds.assign(zeta = (['eta_rho', 'xi_rho'], result))
plt.subplot(122)
# (ds.sel(ocean_time=date)[variable] * ds.mask_rho).plot()
ds.sel(ocean_time=date)[variable].plot()

#### V

In [ ]:
variable = 'v'
ds.isel(ocean_time=-1, s_rho=24)[variable].plot()

In [ ]:
result = (ds[variable] * ds.mask_v.where(ds.mask_v == 1, np.nan))
ds[variable] = result
ds.isel(ocean_time=-1, s_rho=24)[variable].plot()

#### Save

In [ ]:
new_ds

In [ ]:
new_ds.to_netcdf(roho_folder + new_file)